In [ ]:
import torch
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
from datetime import datetime, timedelta
from model_loader import WeatherModel  # Ton fichier existant

# ============================================================================
# 1. SIMULATION DE DONNÉES D'ENTRÉE (24 HEURES)
# ============================================================================
def generate_dummy_history():
    history = []
    base_time = datetime.now() - timedelta(hours=24)
    for i in range(24):
        history.append({
            'temperature_2m': 20.0 + np.sin(i/4.0) * 5,  # Oscillation thermique
            'relative_humidity_2m': 50.0 + np.cos(i/4.0) * 10,
            'surface_pressure': 1013.0 + np.random.normal(0, 1),
            'wind_speed_10m': 5.0 + np.random.random() * 2,
            'wind_gusts_10m': 7.0 + np.random.random() * 2,
            'precipitation': 0.0,
            'snowfall': 0.0,
            'soil_moisture_0_to_7cm': 0.3,
            'timestamp': base_time + timedelta(hours=i)
        })
    return history

# ============================================================================
# 2. FONCTION DE FEATURE ENGINEERING (POUR XGBOOST)
# ============================================================================
def apply_fe_for_xgboost(pred_dict, lat, lon, scaler_xgb):
    # 1. Création du DataFrame
    df = pd.DataFrame([pred_dict])
    
    # 2. Ajout des colonnes de localisation
    df['latitude'] = lat
    df['longitude'] = lon
    df['occurrence'] = 1  # Valeur par défaut utilisée au training
    
    # 3. Ajout des colonnes temporelles (day_sin, day_cos)
    now = datetime.now()
    day_of_year = now.timetuple().tm_yday
    df['day_sin'] = np.sin(2 * np.pi * day_of_year / 365.0)
    df['day_cos'] = np.cos(2 * np.pi * day_of_year / 365.0)
    
    # 4. Compléter les colonnes météo manquantes (non prédites par le GRU)
    # On utilise des valeurs logiques ou basées sur les prédictions existantes
    df['apparent_temperature'] = df.get('apparent_temperature', df['temperature_2m'])
    df['dew_point_2m'] = df.get('dew_point_2m', df['temperature_2m'] - 2)
    df['rain'] = df.get('rain', df['precipitation']) # Si pas de distinction, rain = precip
    df['snow_depth'] = 0.0
    df['wind_direction_10m'] = 180.0 # Valeur neutre
    df['cloud_cover'] = 50.0
    df['weather_code'] = 0.0
    
    # 5. Calcul des Features Engineering (Strictement identique au training)
    df['wind_chill_factor'] = df['apparent_temperature'] - df['temperature_2m']
    df['temp_humidity_index'] = df['temperature_2m'] * df['relative_humidity_2m'] / 100
    df['pressure_anomaly'] = df['surface_pressure'] - 1013.25
    df['lat_lon_interaction'] = df['latitude'] * df['longitude']
    df['distance_from_equator'] = np.abs(df['latitude'])
    df['is_coastal'] = (np.minimum(np.abs(df['longitude'] + 120), 
                                    np.abs(df['longitude'] + 75)) < 10).astype(int)
    df['lat_squared'] = df['latitude'] ** 2
    df['lon_squared'] = df['longitude'] ** 2
    df['lat_abs_lon'] = df['latitude'] * np.abs(df['longitude'])
    df['lat_lon_ratio'] = df['latitude'] / (np.abs(df['longitude']) + 0.001)
    df['snow_to_precip_ratio'] = df['snowfall'] / (df['precipitation'] + 0.001)
    df['lat_temp_interaction'] = df['latitude'] * df['temperature_2m']

    # 6. Sélection et réorganisation automatique
    # On ne garde QUE les colonnes que le scaler connaît, dans le bon ordre
    X_input = df[scaler_xgb.feature_names_in_]
    
    return scaler_xgb.transform(X_input)
# ============================================================================
# 3. MAIN TEST
# ============================================================================
if __name__ == "__main__":
    # --- CHARGEMENT GRU ---
    print("⏳ Chargement du GRU...")
    gru_manager = WeatherModel()
    gru_manager.load_model(
        model_path="../models/gru_best.pth",
        scaler_x_path="../models/scaler_x_transformer.pkl",
        scaler_y_path="../models/scaler_y_transformer.pkl"
    )

    # --- CHARGEMENT XGBOOST ---
    print("⏳ Chargement du Classifier XGBoost...")
    xgb_model = xgb.XGBClassifier()
    xgb_model = joblib.load("../classifier/XGBoost.pkl")

# Tes autres lignes restent identiques
    xgb_scaler = joblib.load("../classifier/scaler.pkl")    
    xgb_le = joblib.load("../classifier/label_encoder.pkl")

    # --- DONNÉES DE TEST ---
    lat, lon = 33.58, -7.60  # Casablanca
    history = generate_dummy_history()
    target_time = datetime.now()

    # --- ÉTAPE 1 : PRÉDICTION GRU ---
    print("\n🚀 Etape 1 : Prédiction GRU (t+1)")
    input_seq = gru_manager.prepare_input_sequence(history, target_time, lat, lon)
    gru_pred = gru_manager.predict(input_seq)
    print(f"   -> Température prédite : {gru_pred['temperature_2m']:.2f}°C")
    print(f"   -> Précipitations : {gru_pred['precipitation']:.2f} mm")

    # --- ÉTAPE 2 : CLASSIFICATION XGBOOST ---
    print("\n🚀 Etape 2 : Classification de l'événement")
    # On passe le résultat du GRU au Feature Engineering du XGBoost
    X_for_xgb = apply_fe_for_xgboost(gru_pred, lat, lon, xgb_scaler)
    
    event_idx = xgb_model.predict(X_for_xgb)[0]
    event_name = xgb_le.inverse_transform([event_idx])[0]
    
    # Optionnel : voir les probabilités
    probs = xgb_model.predict_proba(X_for_xgb)[0]
    confidence = np.max(probs)

    print(f"   -> Résultat : {event_name.upper()}")
    print(f"   -> Confiance : {confidence*100:.2f}%")

    print("\n✅ Test terminé avec succès !")

INFO:model_loader:Chargement du modèle depuis ../models/gru_best.pth
INFO:model_loader:Utilisation du device: cpu
INFO:model_loader:Chargement des scalers...
c:\Users\dell\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:model_loader:Chargement des poids du modèle...
INFO:model_loader:✅ Modèle chargé avec succès!
INFO:model_loader:Features: 12, Targets: 8


⏳ Chargement du GRU...
⏳ Chargement du Classifier XGBoost...

🚀 ÉTAPE 1 : SORTIE BRUTE DU GRU
🔹 temperature_2m            : 16.0572
🔹 relative_humidity_2m      : 62.0081
🔹 surface_pressure          : 1004.3950
🔹 wind_speed_10m            : 6.1442
🔹 wind_gusts_10m            : 12.4712
🔹 precipitation             : 0.0355
🔹 snowfall                  : 0.0017
🔹 soil_moisture_0_to_7cm    : 0.2944

🚀 ÉTAPE 2 : FEATURES CALCULÉES POUR XGBOOST
🔸 wind_chill_factor         : 0.0000
🔸 temp_humidity_index       : 9.9567
🔸 pressure_anomaly          : -8.8550
🔸 lat_lon_interaction       : -255.2080
🔸 distance_from_equator     : 33.5800
🔸 is_coastal                : 0.0000
🔸 snow_to_precip_ratio      : 0.0470
🔸 lat_temp_interaction      : 539.1993

🚀 ÉTAPE 3 : CLASSIFICATION FINALE
✅ ÉVÉNEMENT DÉTECTÉ : NO-EVENT
🎯 CONFIANCE         : 97.83%


c:\Users\dell\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
